1 每一个Ag 都会争取向用户问问题, 每个Ag代表一个话题, 并维护自己的话题
2 具体用户最近的Ag 会被点名道问问题
3 当用户开始与Ag聊天, 会不断聊天, 只有当用户表现出不耐之后, Ag被清退, \
4 所有的Ag 都在听聊天中
5 被聊完天的人会将被踢出很远


In [ ]:
'''
Author: 823042332@qq.com 823042332@qq.com
Date: 2025-09-01 10:40:22
LastEditors: 823042332@qq.com 823042332@qq.com
LastEditTime: 2025-09-01 10:57:32
FilePath: /dl_interview/src/dl_interview/core.py
Description: 这是默认设置,请设置`customMade`, 打开koroFileHeader查看配置 进行设置: https://github.com/OBKoro1/koro1FileHeader/wiki/%E9%85%8D%E7%BD%AE
'''
# http://localhost:8888/lab/tree/实验/数字人生2/数字人生22.ipynb
# obsidian://open?vault=%E5%B7%A5%E4%BD%9C&file=%E4%BA%8B%E4%BB%B6%E7%9C%8B%E6%9D%BF%2FTODO%2F%E9%97%AE%E9%A2%98.canvas


import numpy as np
import random
import matplotlib.pyplot as plt
import time

          
                    
                    
class Reporter:
    def __init__(self, id, num_reporters, topic_pool):
        """
        num_reporters = 10
        topic_min=0, 
        topic_max=100,
        
        
        topic_pool = ["爱情","事业","美食","行业","工作"]
        """
        self.id = id
        self.话题 = random.sample(["爱情","事业","美食","行业","工作"],1)[0] # 记者的话题位置
        self.准备的问题 = ""
        # self.velocity = random.uniform(-0.1 * (topic_max - topic_min), 0.1 * (topic_max - topic_min)) # 记者调整话题的速度
        # self.pbest_position = self.position # 记者自己最好的提问位置
        # self.pbest_fitness = -float('inf') # 记者自己最好的适应度
        self.当前距离提问者的距离 = 3
        self.chat_history = ""
        self.all_chat_history = ''

        self.is_active = False # 是否正在采访
        self.被踢出后的距离 = 10
        self.cool_down_timer = 0 # 被踢出后冷却计时
        self.interview_duration = 0 # 当前采访时长

        self.聊天历史 = ""
        # self.topic_min = topic_min
        # self.topic_max = topic_max
        self.max_interview_duration = 5

    def calculate_fitness(self, interviewee_interest): # 爱情  10
        """
        计算记者当前的适应度。
        适应度越高，表示记者的话题越接近受访者兴趣，越容易获得采访机会。
        会考虑冷却时间。
        """
        
        # 使用相似度来判断距离
        # 如果记者正在冷却，适应度大幅降低
        # 如果记者刚刚结束采访，也给予一定的惩罚，防止立即再次被选中 (即便没有完全冷却)
        
        if interviewee_interest == self.话题:
            return random.uniform(0.8,1.0)
        else:
            return random.uniform(0.2,0.3)
        

    def update_velocity_and_position(self, gbest_position, interviewee_interest, w=0.5, c1=1.5, c2=1.5):
        """
        更新记者的速度和话题位置。
        w: 惯性权重
        c1: 认知学习因子 (受自己历史最佳影响)
        c2: 社会学习因子 (受全局最佳影响)
        """

        # 更新准备的问题
        self.准备的问题 = "更新了问题"

    def reset_after_interview(self):
        """
        当采访结束（被清退）时，将记者“踢出很远”。
        """
        print(f"  记者 {self.id} 被清退。")
        # 随机重置到一个远离当前兴趣点的位置，或者设置一个冷却计时
        # 简单起见，我们随机重置到话题范围内的任意位置，并启动冷却计时
        self.cool_down_timer = 5 # 冷却5个时间步
        self.is_active = False
        self.interview_duration = 0 # 重置采访时长


def simulate_interview_process(num_reporters=10, max_timesteps=100,
                              topic_min=0, topic_max=100,
                              max_interview_duration=10): # 受访者不耐前的最大持续时间
    # max_interview_duration 最大采访时间
    
    topic_pool = ["爱情","事业","美食","行业","工作"]
    
    
    reporters = [Reporter(i, num_reporters, topic_pool) for i in range(num_reporters)]
    
    interviewee_interest = "爱情" # 受访者初始兴趣点

    gbest_position = (0,0)
    gbest_fitness = -float('inf')

    active_reporter = None
    active_reporter_id = -1

    history = {
        "timestep": [],
        "interviewee_interest": [],
        "reporter_positions": [[] for _ in range(num_reporters)],
        "active_reporter_id": [],
        "gbest_position": []
    }

    print(f"--- 模拟开始 ---")
    print(f"初始受访者兴趣点: {interviewee_interest}")

    for t in range(max_timesteps):
        print(f"\n--- 时间步 {t+1} ---")
        history["timestep"].append(t+1)
        history["interviewee_interest"].append(interviewee_interest)
        history["active_reporter_id"].append(active_reporter_id)
        history["gbest_position"].append(gbest_position)

        # 1. 如果没有记者在采访，选择一个记者
        if active_reporter is None:
            best_reporter_for_interview = None
            max_current_fitness = -float('inf')

            # 更新所有记者的pBest并找到全局最佳
            for reporter in reporters:
                current_fitness = reporter.calculate_fitness(interviewee_interest, max_interview_duration)
                # 寻找当前时间步下，最适合采访的记者（排除冷却中的）
                if reporter.cool_down_timer == 0 and current_fitness > max_current_fitness:
                    max_current_fitness = current_fitness
                    best_reporter_for_interview = reporter

            # 选择最适合的记者进行采访
            if best_reporter_for_interview:
                active_reporter = best_reporter_for_interview
                active_reporter.is_active = True
                active_reporter_id = active_reporter.id
                print(f"受访者选择了记者 {active_reporter_id} (话题: {active_reporter.话题}) 进行采访。")
            else:
                print("所有记者都在冷却中，无人可采访。")
        # 2. 模拟采访过程或冷却倒计时
        
        if active_reporter:
            active_reporter.interview_duration += 1
            print(f"  记者 {active_reporter_id} 正在采访... (已持续 {active_reporter.interview_duration} 步)")
            if active_reporter.interview_duration >= max_interview_duration:
                # TODO 记者采访
                print(f"  受访者对记者 {active_reporter_id} 表现出不耐！")
                # 判断是否耐烦 如果不耐

                active_reporter.reset_after_interview()
                active_reporter = None
                active_reporter_id = -1
        else:
            # 冷却计时器递减
            for reporter in reporters:
                if reporter.cool_down_timer > 0:
                    reporter.cool_down_timer -= 1
                    if reporter.cool_down_timer == 0:
                        print(f"  记者 {reporter.id} 冷却期结束，准备再次争取采访。")
        
        # TODO 
        # 3. 所有记者（包括不活跃的）都在“听”，并更新自己的位置
        当前的话题 = -float('inf')
        当前的聊天历史 = gbest_position # 默认不变 current_gbest_position

        

        # 遍历所有记者，根据采访信息调整gbest
        # 如果有活跃记者，其话题可以作为gbest的参考
        if active_reporter and active_reporter.is_active: # 正在采访的记者
             # 假设当前活跃记者的位置是最好的，因为它成功获得了采访
            当前的聊天历史 = active_reporter.聊天历史
            当前的话题 = active_reporter.calculate_fitness(interviewee_interest, max_interview_duration)
        else: # 没有活跃记者，或者活跃记者刚刚被清退，gbest可能需要重新评估
            # 寻找当前所有记者中pBest最好的作为gBest候选
            for reporter in reporters:
                if reporter.话题得分 > 当前的话题:
                    当前的话题 = reporter.话题
                    当前的聊天历史 = reporter.聊天历史

        # 更新全局最佳
        if 当前的话题 > gbest_fitness:
            gbest_fitness = 当前的话题
            gbest_position = 当前的聊天历史
            print(f"  全局最佳话题 (gBest) 更新为: {gbest_position:.2f}")


        for reporter in reporters:
            reporter.update_velocity_and_position(gbest_position, interviewee_interest)
            history["reporter_positions"][reporter.id].append(reporter.position)
            print(f"  记者 {reporter.id}: 位置 {reporter.position:.2f}, 冷却 {reporter.cool_down_timer}")

        # 受访者兴趣点可能随时间微调


        # 您是这样一种人工智能助手，旨在根据用户的聊天意图来更新话题得分，
        # 您的主要任务是识别用户的聊天意图，并根据对话内容更新这些分数, 这些分数代表, 未来用户的话题倾向性.
        # 至关重要的是，您不应向用户说明分数变化的原因。只需展示更新后的分数即可
        # 分数的更新应基于对用户提供的整个对话历史的分析，而非单个回合的内容。用户会为您提供需要分析的对话记录
        # 并据此进行操作。您的操作应遵循以下原则：

        #     * 初始数据表：json
        #     { “爱情”：0.6, "事业": 0.6, "美食": 0.6, "行业": 0.6. 
        # “工作”：0.6 
        # }

        interviewee_interest = judge(当前的聊天历史)

    print(f"\n--- 模拟结束 ---")
    return history, reporters


# 运行模拟
history, final_reporters = simulate_interview_process(num_reporters=10, max_timesteps=50, max_interview_duration=5)

# 可视化结果
plt.figure(figsize=(15, 8))

# 绘制记者位置随时间的变化
for i in range(len(history["reporter_positions"])):
    plt.plot(history["timestep"], history["reporter_positions"][i], label=f'Reporter {i} Position', alpha=0.6)

# 绘制受访者兴趣点
plt.plot(history["timestep"], history["interviewee_interest"], linestyle='--', color='black', label='Interviewee Interest')

# 绘制全局最佳位置
plt.plot(history["timestep"], history["gbest_position"], linestyle=':', color='red', label='Global Best Position (gBest)', linewidth=2)

# 标记活跃记者
for t, active_id in enumerate(history["active_reporter_id"]):
    if active_id != -1:
        # plt.scatter(history["timestep"][t], history["reporter_positions"][active_id][t], color='red', marker='o', s=50, zorder=5)
        # plt.annotate(f'Active {active_id}', (history["timestep"][t], history["reporter_positions"][active_id][t]), textcoords="offset points", xytext=(0,10), ha='center', fontsize=8, color='red')
        pass # 暂时不画散点，太密集

plt.title('Reporter Topic Positions Over Time (Simulating Interview Process)')
plt.xlabel('Time Step')
plt.ylabel('Topic Position (0-100)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# 打印最终记者状态
print("\n--- 最终记者状态 ---")
for reporter in final_reporters:
    print(f"记者 {reporter.id}: 最终位置 {reporter.position:.2f}, pBest {reporter.pbest_position:.2f}, 冷却 {reporter.cool_down_timer}")
print(f"最终全局最佳话题 (gBest): {history['gbest_position'][-1]:.2f}")



In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time


class Reporter:
    def __init__(self, id, num_reporters, topic_min, topic_max):
        """
        num_reporters = 10
        topic_min=0, 
        topic_max=100,
        
        
        """
        self.id = id
        self.position = random.uniform(topic_min, topic_max) # 记者的话题位置
        self.velocity = random.uniform(-0.1 * (topic_max - topic_min), 0.1 * (topic_max - topic_min)) # 记者调整话题的速度
        self.pbest_position = self.position # 记者自己最好的提问位置
        self.pbest_fitness = -float('inf') # 记者自己最好的适应度
        self.is_active = False # 是否正在采访
        self.cool_down_timer = 0 # 被踢出后冷却计时
        self.interview_duration = 0 # 当前采访时长
        self.topic_min = topic_min
        self.topic_max = topic_max

    def calculate_fitness(self, interviewee_interest, max_interview_duration):
        """
        计算记者当前的适应度。
        适应度越高，表示记者的话题越接近受访者兴趣，越容易获得采访机会。
        会考虑冷却时间。
        """
        # 话题距离适应度：越近越高
        distance = abs(self.position - interviewee_interest)
        topic_fitness = max(0, self.topic_max - self.topic_min - distance) # 距离越小，值越大

        # 如果记者正在冷却，适应度大幅降低
        if self.cool_down_timer > 0:
            return topic_fitness * 0.01 # 冷却期间适应度很低

        # 如果记者刚刚结束采访，也给予一定的惩罚，防止立即再次被选中 (即便没有完全冷却)
        if self.interview_duration > 0 and not self.is_active: # 刚刚被清退但冷却期未过
            return topic_fitness * 0.05 # 给予一定惩罚

        return topic_fitness

    def update_velocity_and_position(self, gbest_position, interviewee_interest, w=0.5, c1=1.5, c2=1.5):
        """
        更新记者的速度和话题位置。
        w: 惯性权重
        c1: 认知学习因子 (受自己历史最佳影响)
        c2: 社会学习因子 (受全局最佳影响)
        """
        r1 = random.random()
        r2 = random.random()

        # 计算新的速度
        cognitive_component = c1 * r1 * (self.pbest_position - self.position)
        social_component = c2 * r2 * (gbest_position - self.position)
        self.velocity = w * self.velocity + cognitive_component + social_component

        # 限制速度，防止过冲
        max_vel = 0.2 * (self.topic_max - self.topic_min)
        self.velocity = max(-max_vel, min(max_vel, self.velocity))

        # 更新位置
        self.position += self.velocity

        # 限制位置在话题范围内
        self.position = max(self.topic_min, min(self.topic_max, self.position))

    def reset_after_interview(self):
        """
        当采访结束（被清退）时，将记者“踢出很远”。
        """
        print(f"  记者 {self.id} 被清退。")
        # 随机重置到一个远离当前兴趣点的位置，或者设置一个冷却计时
        # 简单起见，我们随机重置到话题范围内的任意位置，并启动冷却计时
        self.position = random.uniform(self.topic_min, self.topic_max)
        self.cool_down_timer = 5 # 冷却5个时间步
        self.is_active = False
        self.interview_duration = 0 # 重置采访时长


def simulate_interview_process(num_reporters=10, max_timesteps=100,
                              topic_min=0, topic_max=100,
                              max_interview_duration=10): # 受访者不耐前的最大持续时间

    reporters = [Reporter(i, num_reporters, topic_min, topic_max) for i in range(num_reporters)]
    interviewee_interest = random.uniform(topic_min, topic_max) # 受访者初始兴趣点

    gbest_position = random.uniform(topic_min, topic_max)
    gbest_fitness = -float('inf')

    active_reporter = None
    active_reporter_id = -1

    history = {
        "timestep": [],
        "interviewee_interest": [],
        "reporter_positions": [[] for _ in range(num_reporters)],
        "active_reporter_id": [],
        "gbest_position": []
    }

    print(f"--- 模拟开始 ---")
    print(f"初始受访者兴趣点: {interviewee_interest:.2f}")

    for t in range(max_timesteps):
        print(f"\n--- 时间步 {t+1} ---")
        history["timestep"].append(t+1)
        history["interviewee_interest"].append(interviewee_interest)
        history["active_reporter_id"].append(active_reporter_id)
        history["gbest_position"].append(gbest_position)

        # 1. 如果没有记者在采访，选择一个记者
        if active_reporter is None:
            best_reporter_for_interview = None
            max_current_fitness = -float('inf')

            # 更新所有记者的pBest并找到全局最佳
            for reporter in reporters:
                current_fitness = reporter.calculate_fitness(interviewee_interest, max_interview_duration)
                if current_fitness > reporter.pbest_fitness:
                    reporter.pbest_fitness = current_fitness
                    reporter.pbest_position = reporter.position

                # 寻找当前时间步下，最适合采访的记者（排除冷却中的）
                if reporter.cool_down_timer == 0 and current_fitness > max_current_fitness:
                    max_current_fitness = current_fitness
                    best_reporter_for_interview = reporter

            # 选择最适合的记者进行采访
            if best_reporter_for_interview:
                active_reporter = best_reporter_for_interview
                active_reporter.is_active = True
                active_reporter_id = active_reporter.id
                print(f"受访者选择了记者 {active_reporter_id} (话题: {active_reporter.position:.2f}) 进行采访。")
            else:
                print("所有记者都在冷却中，无人可采访。")

        # 2. 模拟采访过程或冷却倒计时
        if active_reporter:
            active_reporter.interview_duration += 1
            print(f"  记者 {active_reporter_id} 正在采访... (已持续 {active_reporter.interview_duration} 步)")
            if active_reporter.interview_duration >= max_interview_duration:
                print(f"  受访者对记者 {active_reporter_id} 表现出不耐！")
                active_reporter.reset_after_interview()
                active_reporter = None
                active_reporter_id = -1
        else:
            # 冷却计时器递减
            for reporter in reporters:
                if reporter.cool_down_timer > 0:
                    reporter.cool_down_timer -= 1
                    if reporter.cool_down_timer == 0:
                        print(f"  记者 {reporter.id} 冷却期结束，准备再次争取采访。")

        # 3. 所有记者（包括不活跃的）都在“听”，并更新自己的位置
        current_gbest_fitness = -float('inf')
        current_gbest_position = gbest_position # 默认不变

        # 遍历所有记者，根据采访信息调整gbest
        # 如果有活跃记者，其话题可以作为gbest的参考
        if active_reporter and active_reporter.is_active: # 正在采访的记者
             # 假设当前活跃记者的位置是最好的，因为它成功获得了采访
            current_gbest_position = active_reporter.position
            current_gbest_fitness = active_reporter.calculate_fitness(interviewee_interest, max_interview_duration)
        else: # 没有活跃记者，或者活跃记者刚刚被清退，gbest可能需要重新评估
            # 寻找当前所有记者中pBest最好的作为gBest候选
            for reporter in reporters:
                if reporter.pbest_fitness > current_gbest_fitness:
                    current_gbest_fitness = reporter.pbest_fitness
                    current_gbest_position = reporter.pbest_position

        # 更新全局最佳
        if current_gbest_fitness > gbest_fitness:
            gbest_fitness = current_gbest_fitness
            gbest_position = current_gbest_position
            print(f"  全局最佳话题 (gBest) 更新为: {gbest_position:.2f}")


        for reporter in reporters:
            reporter.update_velocity_and_position(gbest_position, interviewee_interest)
            history["reporter_positions"][reporter.id].append(reporter.position)
            print(f"  记者 {reporter.id}: 位置 {reporter.position:.2f}, 冷却 {reporter.cool_down_timer}")

        # 受访者兴趣点可能随时间微调
        # interviewee_interest += random.uniform(-1, 1) # 模拟受访者兴趣漂移
        # interviewee_interest = max(topic_min, min(topic_max, interviewee_interest))

    print(f"\n--- 模拟结束 ---")
    return history, reporters


# 运行模拟
history, final_reporters = simulate_interview_process(num_reporters=10, max_timesteps=50, max_interview_duration=5)

# 可视化结果
plt.figure(figsize=(15, 8))

# 绘制记者位置随时间的变化
for i in range(len(history["reporter_positions"])):
    plt.plot(history["timestep"], history["reporter_positions"][i], label=f'Reporter {i} Position', alpha=0.6)

# 绘制受访者兴趣点
plt.plot(history["timestep"], history["interviewee_interest"], linestyle='--', color='black', label='Interviewee Interest')

# 绘制全局最佳位置
plt.plot(history["timestep"], history["gbest_position"], linestyle=':', color='red', label='Global Best Position (gBest)', linewidth=2)

# 标记活跃记者
for t, active_id in enumerate(history["active_reporter_id"]):
    if active_id != -1:
        # plt.scatter(history["timestep"][t], history["reporter_positions"][active_id][t], color='red', marker='o', s=50, zorder=5)
        # plt.annotate(f'Active {active_id}', (history["timestep"][t], history["reporter_positions"][active_id][t]), textcoords="offset points", xytext=(0,10), ha='center', fontsize=8, color='red')
        pass # 暂时不画散点，太密集

plt.title('Reporter Topic Positions Over Time (Simulating Interview Process)')
plt.xlabel('Time Step')
plt.ylabel('Topic Position (0-100)')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True)
plt.tight_layout()
plt.show()

# 打印最终记者状态
print("\n--- 最终记者状态 ---")
for reporter in final_reporters:
    print(f"记者 {reporter.id}: 最终位置 {reporter.position:.2f}, pBest {reporter.pbest_position:.2f}, 冷却 {reporter.cool_down_timer}")
print(f"最终全局最佳话题 (gBest): {history['gbest_position'][-1]:.2f}")

In [ ]:
# 上来就直接问答, 如果遇到需要更换话题的时候, 就请求更换话题和